## Recorde the different mouvement for tictactoe 2021

In [ ]:
from reachy_sdk import ReachySDK
import numpy as np
import time 
from reachy_sdk.trajectory import goto
from reachy_sdk.trajectory.interpolation import InterpolationMode

In [ ]:
reachy = ReachySDK('localhost')

To test the mouvement with your configuration : 

For simple mouvement :

In [ ]:
def goto_position(path): 
        reachy.turn_on('r_arm')
        move = np.load(path)
        move.allow_pickle=1
    
        listMoves = move['move'].tolist()
        print(type(list(listMoves.items())[0]))
        listTraj = { 'self.'+key:float(val) for key,val in listMoves.items() }
        print(listTraj)
    
        goto(
            goal_positions=listTraj, 
            duration=2.0,
            interpolation_mode=InterpolationMode.MINIMUM_JERK
        )

In [ ]:
grab_index=1
path = '/home/reachy/dev/reachy-tictactoe/reachy_tictactoe/moves-2021/shuffle-board.npz'
goto_position(path)

For complexe trajectories : 

In [ ]:
def trajectoryPlayer(path):
    reachy.turn_on('r_arm')
    move = np.load(path)
    move.allow_pickle=1
    print(list(move.keys()))

    listMoves = move['move'].tolist()
    print(listMoves)
    listTraj = [ val for key,val in listMoves.items()]
    listTraj = np.array(listTraj).T.tolist()

    sampling_frequency = 100  #en hertz

    recorded_joints = [
        reachy.r_arm.r_shoulder_pitch,
        reachy.r_arm.r_shoulder_roll,
        reachy.r_arm.r_arm_yaw,
        reachy.r_arm.r_elbow_pitch,
        reachy.r_arm.r_forearm_yaw,
        reachy.r_arm.r_wrist_pitch,
        reachy.r_arm.r_wrist_roll,
        reachy.r_arm.r_gripper, 
    ]

    for joint in recorded_joints:
        joint.compliant = False

    first_point = dict(zip(recorded_joints, listTraj[0]))
    goto(first_point, duration=3.0)

    for joints_positions in listTraj:
        for joint, pos in zip(recorded_joints, joints_positions):
            joint.goal_position = pos
        time.sleep(1 / sampling_frequency)

In [ ]:
path = '/home/reachy/dev/reachy-tictactoe/reachy_tictactoe/moves-2021/shuffle-board.npz'
trajectoryPlayer(path)

Record a new mouvement : 

In [2]:
reachy.turn_off('reachy')

NameError: name 'reachy' is not defined

In [ ]:
trajectories = [] #on créer une nouvelle liste trajectoire

start = time.time() #seconde passées depuis epoch
while (time.time() - start) < record_duration:
    #on optient les positions actuelles de toutes les jointures 
    current_point = [joint.present_position for joint in recorded_joints]
    #on ajoute les positions à la liste trajectoire 
    trajectories.append(current_point)

    time.sleep(1 / sampling_frequency)

Save as a .npz format : 

In [ ]:
np.savez('/home/reachy/dev/reachy-tictactoe/reachy_tictactoe/moves/test',trajectories)